# Chapter 9  
## Metaprogramming

One of the most important mantras of software development is "don’t repeat yourself."  
That is, any time you are faced with a problem of creating highly repetitive code (or cutting and pasting source code), it often pays to look for a more elegant solution.  
In Python, such problems are often solved under the category of "metaprogramming."  
In a nutshell, metaprogramming is about creating functions and classes whose main goal is to manipulate code by modifying, generating, or wrapping existing code.  
The main features for this include decorators, class decorators, and metaclasses.  
However, a variety of other useful topics -- including signature objects, execution of code with exec(), and inspecting the internals of functions and classes -- enter the picture.  
The main purpose of this chapter is to explore various metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.

## 9.1 Putting a Wrapper Around a Function

### Problem

You want to put a wrapper layer around a function that adds extra processing, such as logging, timing, and so forth.

### Solution

You can define a decorator function to wrap a function with extra code.

In [22]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Now let's make our decorator useful.

In [23]:
@timethis
def countdown(n):
    """
    Yes, just a countdown.
    """
    while n > 0:
        n -= 1

In [24]:
countdown(10000)

countdown 0.0005660057067871094


In [25]:
countdown(100000)

countdown 0.007002115249633789


### Discussion

A decorator is a function that accepts a function as input and returns a new function as output.  
Whenever you write code like the following:

In [26]:
@timethis
def countdown(n):
    '''do something'''

it's the same as if you had executed these separate steps:

In [27]:
def countdown(n):
    '''...'''
countdown = timethis(countdown)

Python's built-in decorators, like `@staticmethod`, `@classmethod`, and `@property` work in the same way.  
For example, the following two code fragments will give an equivalent result:

In [28]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    # Equivalent definition of a class method.
    def method(cls):
        pass
    method = classmethod(method)

The code inside a decorator typically involves creating a new function that accepts any arguments using `*args` and `**kwargs`, as shown with the `wrapper()` function in this recipe.  
Inside this function, you place a call to the original input function and return its result.  
However, you also place whatever extra code you want to add, like our timing function.  
The newly created function wrapper is returned as a result and takes the place of the original function.  
It’s critical to emphasize that decorators generally do not alter the calling signature or return value of the function being wrapped.  
The use of `*args` and `**kwargs` is there to make sure that any input arguments can be accepted.  
The return value of a decorator is almost always the result of calling `func(*args, **kwargs)`, where `func` is the original unwrapped function.  
When first learning about decorators, it is usually very easy to get started with some simple examples, such as the one shown.  
However, if you are going to write decorators for real, there are some subtle details to consider.  
For example, the use of the decorator `@wraps(func)` in the solution is an easy to forget but important technicality related to preserving function metadata, which is described in the next recipe.  
The next few recipes that follow fill in some details that will be important if you wish to write decorator functions of your own.

## 9.2. Preserving Function Metadata When Writing Decorators

### Problem

You've written a decorator, but when you apply it to a function, important metadata such as the name, docstring, annotations, and calling signature are lost.

### Solution

Whenever you define a decorator, you should always remember to apply the `@wraps` decorator from the `functools` library to the underlying wrapper function.

In [29]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Let's use the decorator and examine the resulting function metadata:

In [30]:
@timethis
def countdown(n:int):
    """
    Countdown the integers.
    """
    while n > 0:
        n -= 1

In [31]:
countdown(10000)

countdown 0.0006442070007324219


In [32]:
countdown(100000)

countdown 0.0069179534912109375


In [33]:
countdown.__name__

'countdown'

In [34]:
countdown.__doc__

'\n    Countdown the integers.\n    '

In [35]:
countdown.__annotations__

{'n': int}

### Discussion

Copying decorator metadata is an important part of writing decorators.  
If you forget to use `@wraps`, you’ll find that the decorated function loses all sorts of useful information.  
For instance, if omitted, the metadata in the last example would look like this:

An important feature of the `@wraps` decorator is that it makes the wrapped function available to you in the `__wrapped__` attribute.  
For example, if you want to access the wrapped function directly, you could do this:

The presence of the `__wrapped__` attribute also makes decorated functions properly expose the underlying signature of the wrapped function.

In [36]:
from inspect import signature
print(signature(countdown))

(n:int)


One common question that sometimes arises is how to make a decorator that directly copies the calling signature of the original function being wrapped (as opposed to using `*args` and `**kwargs`).  
In general, this is difficult to implement without resorting to some trick involving the generator of code strings and `exec()`.  
Frankly, you’re usually best off using `@wraps` and relying on the fact that the underlying function signature can be propagated by access to the underlying `__wrapped__` attribute.  
See Recipe 9.16 for more information about signatures.

## 9.3. Unwrapping a Decorator

### Problem

A decorator has been applied to a function, but you want to overrride it and gain access to the original unwrapped function.

### Solution

Assuming that the decorator has been implemented properly using `@wraps` (see Recipe 9.2), you can usually gain access to the original function by accessing the `__wrapped__` attribute.

### Discussion

Gaining direct access to the unwrapped function behind a decorator can be useful for debugging, introspection, and other operations involving functions.  
However, this recipe only works if the implementation of a decorator properly copies metadata using `@wraps` from the functools module or sets the `__wrapped__` attribute directly.  
If multiple decorators have been applied to a function, the behavior of accessing `__wrapped__` is currently undefined and should probably be avoided.  
In Python 3.3, it bypasses all of the layers.  
For example, suppose you have code like this:

In [37]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

In [38]:
@decorator1
@decorator2
def add(x, y):
    return x + y

Now let's call the decorated function and the original function through `__wrapped__`.

In [39]:
add(2, 3)

Decorator 1
Decorator 2


5

In [40]:
add.__wrapped__(2, 3)

Decorator 2


5

However, this behavior has been reported as a bug (see `http://bugs.python.org/issue17482`) and may be changed to expose the proper decorator chain in a future release.
Last, but not least, be aware that not all decorators utilize `@wraps`, and thus, they may not work as described.  
In particular, the built-in decorators `@staticmethod` and `@classmethod` create descriptor objects that don’t follow this convention (instead, they store the original function in a `__func__` attribute).  
Your mileage may vary.

## 9.4. Defining a Decorator That Takes Arguments